In [2]:
#coding=utf-8
from keras.models import Model
from keras.layers import Input,Dense,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,ZeroPadding2D
from keras.layers import add,Flatten
#from keras.layers.convolutional import Conv2D,MaxPooling2D,AveragePooling2D
from keras.optimizers import SGD
import numpy as np
seed = 7
np.random.seed(seed)

In [3]:
num_BN = 1
def Conv2d_BN(x, nb_filter,kernel_size, strides=(1,1), padding='same',name=None):
    if name is None:
        name = str(num_BN)
        num_BN += 1 
     
    conv_name = name + "_Conv2D" 
    bn_name = name + "_BN"  
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=conv_name)(x)
    x = BatchNormalization(axis=3,name=bn_name)(x)
    return x
 
def Conv_Block(inpt,nb_filter,kernel_size,strides=(1,1), with_conv_shortcut=False, block_no=None):
    block_name = "block"+str(block_no)
    x = Conv2d_BN(inpt,nb_filter=nb_filter[0],kernel_size=(1,1),strides=strides,padding='same', name=block_name+"_1")
    x = Conv2d_BN(x, nb_filter=nb_filter[1], kernel_size=(3,3), padding='same', name=block_name+"_2")
    x = Conv2d_BN(x, nb_filter=nb_filter[2], kernel_size=(1,1), padding='same', name=block_name+"_3")
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt,nb_filter=nb_filter[2],strides=strides,kernel_size=kernel_size, name="short_block"+str(block_no))
        x = add([x,shortcut])
        return x
    else:
        x = add([x,inpt])
        return x

In [7]:
inpt = Input(shape=(224,224,3), name="Input")
x = ZeroPadding2D((3,3), name="ZeroPadding")(inpt)
x = Conv2d_BN(x,nb_filter=64,kernel_size=(7,7),strides=(2,2),padding='valid', name="NB1") 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same',  name="pool1")(x)
 
x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3),strides=(1,1),with_conv_shortcut=True, block_no=1)
x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3), block_no=2)
x = Conv_Block(x,nb_filter=[64,64,256],kernel_size=(3,3), block_no=3)
 
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True, block_no=4)
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3), block_no=5)
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3), block_no=6)
x = Conv_Block(x,nb_filter=[128,128,512],kernel_size=(3,3), block_no=7)
 
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True, block_no=8)
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3), block_no=9)
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3), block_no=10)
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3), block_no=11)
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3), block_no=12)
x = Conv_Block(x,nb_filter=[256,256,1024],kernel_size=(3,3), block_no=13)
 
x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True, block_no=14)
x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3), block_no=15)
x = Conv_Block(x,nb_filter=[512,512,2048],kernel_size=(3,3), block_no=16)
x = AveragePooling2D(pool_size=(7,7), name="AveragePooling")(x)
x = Flatten(name="Flat1")(x)
x = Dense(1000,activation='softmax', name="Dense1")(x)
 
model = Model(inputs=inpt,outputs=x)
sgd = SGD(decay=0.0001,momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
model.summary() 

# 把模型保存为图片
from keras.utils import plot_model
plot_model(model,to_file='model_png/207Resnet50.png', show_layer_names=True, show_shapes=True) 

W0808 17:11:47.285294 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
ZeroPadding (ZeroPadding2D)     (None, 230, 230, 3)  0           Input[0][0]                      
__________________________________________________________________________________________________
NB1_Conv2D (Conv2D)             (None, 112, 112, 64) 9472        ZeroPadding[0][0]                
__________________________________________________________________________________________________
NB1_BN (BatchNormalization)     (None, 112, 112, 64) 256         NB1_Conv2D[0][0]                 
__________________________________________________________________________________________________
pool1 (Max